# Notebook 3: Experimento Completo de Modelado y Evaluación

Este notebook orquesta un pipeline de modelado de principio a fin, incluyendo:
1. **Carga y Preprocesamiento de Datos en Memoria**: Carga los datos crudos y aplica el pipeline de transformación completo.
2. **Búsqueda de Hiperparámetros**: Utiliza Grid Search manual para encontrar los mejores parámetros para cada modelo.
3. **Entrenamiento y Calibración**: Entrena cada modelo con sus parámetros óptimos y calibra el SVM.
4. **Evaluación Exhaustiva**: Evalúa los modelos finales en el conjunto de prueba usando un conjunto completo de métricas.
5. **Comparación de Resultados**: Presenta una tabla resumen para comparar el rendimiento de todos los modelos.

In [7]:
import sys
import os
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
import time

# Añadir la carpeta 'src' al path
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

# Módulos del proyecto
from data.loader import cargar_csv_crudo
from features.transformers_manual import ImputadorNumericoMediana, ImputadorCategoricoModa, EscaladorEstandarManual, OneHotManual
from models.algorithms_manual import RegresionLogisticaManual, RandomForestManual, SVMManual
from models.knn_wrapper import crear_pipeline_knn
from evaluation.cv_manual import StratifiedKFoldManual, grid_search_manual
from evaluation.metrics_manual import (
    matriz_confusion_manual, balanced_accuracy_manual, precision_recall_f1_por_clase, 
    macro_f1_score, weighted_f1_score, roc_auc_manual, pr_auc_manual
)

## Funciones Auxiliares de Visualización

In [8]:
def graficar_matriz_confusion(matriz_conf, etiquetas_clases, nombre_modelo):
    """Genera un heatmap de la matriz de confusión."""
    plt.figure(figsize=(8, 6))
    sns.heatmap(matriz_conf, annot=True, fmt='d', cmap='Blues',
                xticklabels=etiquetas_clases, yticklabels=etiquetas_clases)
    plt.title(f'Matriz de Confusión para {nombre_modelo}')
    plt.ylabel('Etiqueta Verdadera')
    plt.xlabel('Etiqueta Predicha')
    plt.show()

## 1. Pipeline de Carga y Preprocesamiento de Datos

In [ ]:
def pipeline_preprocesamiento_completo():
    # Carga
    df_train = cargar_csv_crudo('datos_entrenamiento_riesgo.csv')
    df_test = cargar_csv_crudo('datos_prueba_riesgo.csv')
    
    # Separación X, y
    X_train_raw = df_train.drop('nivel_riesgo', axis=1)
    y_train_raw = df_train['nivel_riesgo']
    X_test_raw = df_test.drop('nivel_riesgo', axis=1)
    y_test_raw = df_test['nivel_riesgo']
    
    # Codificar Target
    mapa_target = {'Bajo': 0, 'Medio': 1, 'Alto': 2}
    y_train = y_train_raw.map(mapa_target).values
    y_test = y_test_raw.map(mapa_target).values
    
    # Imputación
    num_cols = list(X_train_raw.select_dtypes(include=np.number).columns)
    cat_cols = list(X_train_raw.select_dtypes(include='object').columns)

    imputador_num = ImputadorNumericoMediana(variables=num_cols).fit(X_train_raw)
    X_train_imp = imputador_num.transform(X_train_raw)
    X_test_imp = imputador_num.transform(X_test_raw)

    imputador_cat = ImputadorCategoricoModa(variables=cat_cols).fit(X_train_imp)
    X_train_imp = imputador_cat.transform(X_train_imp)
    X_test_imp = imputador_cat.transform(X_test_imp)
    
    # One-Hot Encoding
    onehot = OneHotManual(variables=cat_cols, drop_last=True).fit(X_train_imp)
    X_train_oh = onehot.transform(X_train_imp)
    X_test_oh = onehot.transform(X_test_imp)
    
    # Alineación de columnas
    train_cols, test_cols = set(X_train_oh.columns), set(X_test_oh.columns)
    for col in (train_cols - test_cols):
        X_test_oh[col] = 0
    X_test_oh = X_test_oh[X_train_oh.columns]
    
    # Escalado
    feature_names = X_train_oh.columns
    escalador = EscaladorEstandarManual().fit(X_train_oh)
    X_train_scaled = escalador.transform(X_train_oh).values
    X_test_scaled = escalador.transform(X_test_oh).values
    
    return X_train_scaled, y_train, X_test_scaled, y_test, feature_names


# Ejecutar el pipeline
X_train, y_train, X_test, y_test, feature_names = pipeline_preprocesamiento_completo()

Cargando datos desde: e:\06. Sexto Ciclo\01. Machine Learning\07. Workspace\06S03. Proyecto 01\06C_Machine_06S01_Clasification\data\raw\datos_entrenamiento_riesgo.csv
Cargando datos desde: e:\06. Sexto Ciclo\01. Machine Learning\07. Workspace\06S03. Proyecto 01\06C_Machine_06S01_Clasification\data\raw\datos_prueba_riesgo.csv


ValueError: El parámetro 'variables' debe ser una lista o tupla.

## 2. Experimento de Modelado

Ahora definimos los modelos, sus rejillas de hiperparámetros y ejecutamos el ciclo de entrenamiento y evaluación.

In [ ]:
modelos_a_probar = {
    'RegresionLogistica': {
        'clase': RegresionLogisticaManual,
        'rejilla': {'tasa_aprendizaje': [0.01, 0.1], 'C': [1, 10], 'epocas': [500]},
        'calibrar': False
    },
    'RandomForest': {
        'clase': RandomForestManual,
        'rejilla': {'n_estimadores': [50, 100], 'max_profundidad': [5, 10]},
        'calibrar': False
    },
    'SVM': {
        'clase': SVMManual,
        'rejilla': {'tasa_aprendizaje': [0.001, 0.01], 'C': [1, 10]},
        'calibrar': True # SVM necesita calibración para probabilidades
    },
    'KNN': {
        'clase': crear_pipeline_knn, # Es una función que devuelve un pipeline
        'rejilla': {'n_neighbors': [5, 11], 'weights': ['uniform', 'distance']},
        'calibrar': False
    }
}

resultados_finales = {}

In [ ]:
for nombre_modelo, config in modelos_a_probar.items():
    print(f"\n{'='*20} PROCESANDO MODELO: {nombre_modelo} {'='*20}")
    
    # Búsqueda de hiperparámetros
    cv = StratifiedKFoldManual(n_splits=3, shuffle=True, random_state=42)
    mejores_params, _ = grid_search_manual(
        estimador_clase=config['clase'],
        rejilla_parametros=config['rejilla'],
        X=X_train, y=y_train, cv=cv
    )
    
    # Entrenamiento del modelo final
    print(f"Entrenando {nombre_modelo} con los mejores parámetros: {mejores_params}")
    modelo_final = config['clase'](**mejores_params)
    modelo_final.fit(X_train, y_train)
    
    # Calibración y predicción de probabilidades
    if config['calibrar']:
        print("Calibrando modelo SVM...")
        scores_calibracion = modelo_final.predict_scores(X_train)
        calibrador = RegresionLogisticaManual(epocas=100).fit(scores_calibracion, y_train)
        scores_test = modelo_final.predict_scores(X_test)
        y_pred_proba = calibrador.predict_proba(scores_test)
        y_pred = np.argmax(y_pred_proba, axis=1)
    else:
        y_pred_proba = modelo_final.predict_proba(X_test)
        y_pred = modelo_final.predict(X_test)
    
    # Evaluación
    etiquetas_clases = ['Bajo', 'Medio', 'Alto']
    matriz_conf, _ = matriz_confusion_manual(y_test, y_pred, etiquetas=[0, 1, 2])
    metricas_clase = precision_recall_f1_por_clase(matriz_conf)
    
    # a. Graficar matriz de confusión
    graficar_matriz_confusion(matriz_conf, etiquetas_clases, nombre_modelo)
    
    # b. Calcular macro precision y recall
    macro_precision = np.mean([v['precision'] for v in metricas_clase.values()])
    macro_recall = np.mean([v['recall'] for v in metricas_clase.values()])
    
    # c. Añadir todas las métricas a los resultados
    resultados_finales[nombre_modelo] = {
        'Precision (Macro)': macro_precision,
        'Recall (Macro)': macro_recall,
        'Balanced Accuracy': balanced_accuracy_manual(matriz_conf),
        'Macro F1-Score': macro_f1_score(metricas_clase),
        'Weighted F1-Score': weighted_f1_score(metricas_clase, matriz_conf),
        'AUC-ROC (Macro)': roc_auc_manual(y_test, y_pred_proba),
        'PR-AUC (Macro)': pr_auc_manual(y_test, y_pred_proba)
    }
    
    print(f"\n--- Resultados para {nombre_modelo} ---")
    print(pd.DataFrame([resultados_finales[nombre_modelo]]).T)
    
    # Guardar modelo
    if not os.path.exists('../models'): os.makedirs('../models')
    joblib.dump(modelo_final, f'../models/{nombre_modelo.lower()}_final.joblib')

## 3. Tabla Comparativa de Resultados

In [ ]:
df_resultados = pd.DataFrame(resultados_finales).T
df_resultados.index.name = 'Modelo'

print("\n" + "="*50)
print("TABLA COMPARATIVA FINAL DE MODELOS")
print("="*50)
display(df_resultados.style.background_gradient(cmap='viridis', axis=0))